In [2]:
import pandas as pd
import time
from ncaa_bbStats import get_team_stat, list_all_teams, get_pythagorean_expectation

START_YEAR = 2002
END_YEAR = 2025
DIVISION = 1
OUTPUT_CSV = "ncaabb_dataset.csv"

# Miami,FL needs to be changed to University of Miami,ACC
# Miami,OH needs to be changed to Miami University-Ohio,MAC
# LMU,CA needs to be changed to LMU,WCC
# Saint Mary's,CA needs to be changed to Saint Mary's,WCC
# St. John's,NY needs to be changed to St. John's,Big East

STAT_NAMES = [
    "W", "L", "T", "WPCT", "PE", "Difference",
    "G", "BB (Batting)", "AB", "H", "BA", "DP", "DPPG", "2B", "2BPG",
    "IP", "R (Pitching)", "ER", "ERA", "PO", "A", "E", "FPCT",
    "HB", "HBP", "HA", "HAPG", "HR", "HRPG", "SF", "SH", "OBP",
    "SB", "SBPG", "CS", "R (Batting)", "RPG", "SHO", "TB", "SLG",
    "SO", "BB (Pitching)", "K/BB", "K/9", "TP", "3B", "3BPG", "WHIP", "BBPG (Pitching)"
]

rows = []

start_time = time.time()

for year in range(START_YEAR, END_YEAR + 1):
    print(f"Processing {year}...")
    try:
        teams = list_all_teams(year, DIVISION)
    except Exception as e:
        print(f"Error getting teams for {year}: {e}")
        continue
    
    for team_full in teams:
        # Split team and league
        if "(" in team_full and ")" in team_full:
            team_name = team_full.split("(")[0].strip()
            league_name = team_full.split("(")[1].replace(")", "").strip()
        else:
            continue
        
        # Skip rows where team/league name is suspiciously short or empty
        if not team_name or not league_name or team_name.lower().startswith("division"):
            continue
        
        row = {"team": team_name, "league": league_name, "year": year}
        
        # Get W, L, T, G first
        try:
            row["W"] = get_team_stat("W", team_full, year, DIVISION)
        except Exception:
            row["W"] = None
        try:
            row["L"] = get_team_stat("L", team_full, year, DIVISION)
        except Exception:
            row["L"] = None
        try:
            row["T"] = get_team_stat("T", team_full, year, DIVISION)
        except Exception:
            row["T"] = None
        try:
            row["G"] = get_team_stat("G", team_full, year, DIVISION)
        except Exception:
            row["G"] = None

        # WPCT from W / G instead of API
        if row["W"] is not None and row["G"] not in (None, 0):
            row["WPCT"] = round(row["W"] / row["G"], 3)  # 3 decimal places
        else:
            row["WPCT"] = None
    
        # Add Pythagorean expectation
        pytha_val = get_pythagorean_expectation(team_name, year, DIVISION)
        def safe_float(val):
            """Convert to float if possible, else return None."""
            try:
                return float(val)
            except (ValueError, TypeError):
                return None
                
        pytha_val = safe_float(pytha_val)
        row["PE"] = round(pytha_val, 3) if pytha_val is not None else None

        # Add difference column (WPCT - PE)
        if row["WPCT"] is not None and row["PE"] is not None:
            row["Difference"] = round(row["WPCT"] - row["PE"], 3)
        else:
            row["Difference"] = None
    
        # Remaining stats
        for stat in STAT_NAMES:
            if stat in ["W", "L", "T", "WPCT", "PE", "Difference"]:
                continue
            try:
                row[stat] = get_team_stat(stat, team_full, year, DIVISION)
            except Exception:
                row[stat] = None
        
        rows.append(row)

df = pd.DataFrame(rows)

# Remove rows where all stats are NaN/0
df = df.dropna(subset=STAT_NAMES, how="all")

# Final safety: remove any stray rows with "division" in team/league
df = df[~df["team"].str.lower().str.contains("division", na=False)]
df = df[~df["league"].str.lower().str.contains("division", na=False)]

# Convert numeric fields
for stat in STAT_NAMES:
    df[stat] = pd.to_numeric(df[stat], errors="coerce")

df.to_csv(OUTPUT_CSV, index=False)
print(f"Dataset saved to {OUTPUT_CSV} with {len(df)} rows after cleanup.")

# Print total runtime
elapsed_time = time.time() - start_time
print(f"Script completed in {elapsed_time:.2f} seconds.")

Processing 2002...
Processing 2003...
Processing 2004...
Processing 2005...
Processing 2006...
Processing 2007...
Processing 2008...
Processing 2009...
Processing 2010...
Processing 2011...
Processing 2012...
Processing 2013...
Processing 2014...
Processing 2015...
Processing 2016...
Processing 2017...
Processing 2018...
Processing 2019...
Processing 2020...
Processing 2021...
Processing 2022...
Processing 2023...
Processing 2024...
Processing 2025...
Dataset saved to ncaabb_dataset.csv with 6787 rows after cleanup.
Script completed in 744.11 seconds.


In [3]:
import numpy as np
import pandas as pd

INPUT_CSV = "ncaabb_dataset.csv"

# columns in CSV that hold runs scored and runs allowed
RS_COL = "R (Batting)"    # runs scored
RA_COL = "R (Pitching)"   # runs allowed
WPCT_COL = "WPCT"         # actual win percentage in CSV

# grid search range for exponent
GRID_MIN, GRID_MAX, GRID_STEP = 0.5, 3.5, 0.01

# metric: RMSE
def rmse(y_true, y_pred):
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    if mask.sum() == 0:
        return np.nan
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

df = pd.read_csv(INPUT_CSV)

# ensure numeric
for c in [RS_COL, RA_COL, WPCT_COL]:
    if c not in df.columns:
        raise KeyError(f"Column {c} not found in CSV")
    df[c] = pd.to_numeric(df[c], errors="coerce")

# remove rows with missing RS/RA/WPCT
mask = ~df[RS_COL].isna() & ~df[RA_COL].isna() & ~df[WPCT_COL].isna()
df_clean = df[mask].copy()
# Filter so it's only 2021-2025, matching player_stat years
if "year" in df_clean.columns:
    df_clean = df_clean[(df_clean["year"] >= 2021) & (df_clean["year"] <= 2025)]

print(f"Using {len(df_clean)} rows for fitting (non-missing RS/RA/WPCT).")

# convert to numpy arrays
RS = df_clean[RS_COL].to_numpy(dtype=float)
RA = df_clean[RA_COL].to_numpy(dtype=float)
WPCT = df_clean[WPCT_COL].to_numpy(dtype=float)

# Check which years remain after cleaning
print(df_clean["year"].unique())

# Safety: avoid zero runs (if both zero, skip those rows)
valid = (RS + RA) > 0
RS = RS[valid]; RA = RA[valid]; WPCT = WPCT[valid]
print(f"{len(RS)} rows after removing zero total-run rows.")

# Grid search
grid = np.arange(GRID_MIN, GRID_MAX + 1e-9, GRID_STEP)
best_x = None
best_rmse = np.inf
rmses = []

for x in grid:
    # compute predicted win% for exponent x
    with np.errstate(over='ignore', invalid='ignore'):
        RSx = np.power(RS, x)
        RAx = np.power(RA, x)
        pred = RSx / (RSx + RAx)
    cur_rmse = rmse(WPCT, pred)
    rmses.append(cur_rmse)
    if np.isfinite(cur_rmse) and cur_rmse < best_rmse:
        best_rmse = cur_rmse
        best_x = x

print(f"Grid search best exponent: {best_x:.2f} with RMSE = {best_rmse:.5f}")

# compare to MLB baseline 1.83
mlb_x = 1.83
RSx = np.power(RS, mlb_x); RAx = np.power(RA, mlb_x)
mlb_pred = RSx / (RSx + RAx)
mlb_rmse = rmse(WPCT, mlb_pred)
print(f"Baseline exponent {mlb_x} RMSE = {mlb_rmse:.5f}")
print(f"Improvement % in RMSE: {((mlb_rmse - best_rmse) / mlb_rmse) * 100:.2f}")

# Compute per-year best exponent
compute_per_year = True
per_year_results = {}
if compute_per_year and "year" in df_clean.columns:
    for year, group in df_clean.groupby("year"):
        RSy = group[RS_COL].to_numpy(dtype=float)
        RAy = group[RA_COL].to_numpy(dtype=float)
        WPCTy = group[WPCT_COL].to_numpy(dtype=float)
        valid = (RSy + RAy) > 0
        RSy = RSy[valid]; RAy = RAy[valid]; WPCTy = WPCTy[valid]
        if len(RSy) < 10:  # skip tiny years
            continue
        
        # Find best exponent
        best_x_y = None
        best_rmse_y = np.inf
        for x in grid:
            RSx = np.power(RSy, x)
            RAx = np.power(RAy, x)
            pred = RSx / (RSx + RAx)
            cur_rmse = rmse(WPCTy, pred)
            if np.isfinite(cur_rmse) and cur_rmse < best_rmse_y:
                best_rmse_y = cur_rmse
                best_x_y = x
        
        # Calculate MLB 1.83 RMSE
        RSx_mlb = np.power(RSy, 1.83)
        RAx_mlb = np.power(RAy, 1.83)
        pred_mlb = RSx_mlb / (RSx_mlb + RAx_mlb)
        mlb_rmse_y = rmse(WPCTy, pred_mlb)
        
        # improvement %
        improvement_pct = ((mlb_rmse_y - best_rmse_y) / mlb_rmse_y) * 100
        
        # Results (best exponent, best RMSE, MLB RMSE, improvement %)
        per_year_results[year] = (best_x_y, best_rmse_y, mlb_rmse_y, improvement_pct)

    print("Per-year top exponents (sample):")
    for year in sorted(per_year_results.keys()):
        best_x_y, best_rmse_y, mlb_rmse_y, improvement_pct = per_year_results[year]
        print(
            int(year),
            (
                float(round(best_x_y, 3)),
                float(round(best_rmse_y, 4)),
                1.83,
                float(round(mlb_rmse_y, 4)),
                float(round(improvement_pct, 2))
            )
        )

Using 1468 rows for fitting (non-missing RS/RA/WPCT).
[2021 2022 2023 2024 2025]
1468 rows after removing zero total-run rows.
Grid search best exponent: 1.79 with RMSE = 0.04586
Baseline exponent 1.83 RMSE = 0.04594
Improvement % in RMSE: 0.16
Per-year top exponents (sample):
2021 (1.71, 0.0479, 1.83, 0.0486, 1.4)
2022 (1.77, 0.0443, 1.83, 0.0445, 0.41)
2023 (1.83, 0.0443, 1.83, 0.0443, -0.0)
2024 (1.85, 0.0446, 1.83, 0.0446, 0.02)
2025 (1.8, 0.0476, 1.83, 0.0476, 0.07)


In [5]:
import numpy as np
import pandas as pd

INPUT_CSV = "ncaabb_dataset.csv"

# columns in CSV that hold runs scored and runs allowed
RS_COL = "R (Batting)"    # runs scored
RA_COL = "R (Pitching)"   # runs allowed
WPCT_COL = "WPCT"         # actual win percentage in CSV

# grid search range for exponent
GRID_MIN, GRID_MAX, GRID_STEP = 0.5, 3.5, 0.01

# metric: RMSE
def rmse(y_true, y_pred):
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    if mask.sum() == 0:
        return np.nan
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

df = pd.read_csv(INPUT_CSV)

# ensure numeric
for c in [RS_COL, RA_COL, WPCT_COL]:
    if c not in df.columns:
        raise KeyError(f"Column {c} not found in CSV")
    df[c] = pd.to_numeric(df[c], errors="coerce")

# remove rows with missing RS/RA/WPCT
mask = ~df[RS_COL].isna() & ~df[RA_COL].isna() & ~df[WPCT_COL].isna()
df_clean = df[mask].copy()
# Filter so it's only 2021-2025, matching player_stat years
if "year" in df_clean.columns:
    df_clean = df_clean[(df_clean["year"] >= 2021) & (df_clean["year"] <= 2025)]
    
print(f"Using {len(df_clean)} rows for fitting (non-missing RS/RA/WPCT).")

# convert to numpy arrays
RS = df_clean[RS_COL].to_numpy(dtype=float)
RA = df_clean[RA_COL].to_numpy(dtype=float)
WPCT = df_clean[WPCT_COL].to_numpy(dtype=float)

# Check which years remain after cleaning
print(df_clean["year"].unique())

# safety: avoid zero runs (if both zero, skip those rows)
valid = (RS + RA) > 0
RS = RS[valid]; RA = RA[valid]; WPCT = WPCT[valid]
print(f"{len(RS)} rows after removing zero total-run rows.")

# Grid search
grid = np.arange(GRID_MIN, GRID_MAX + 1e-9, GRID_STEP)
best_x = None
best_rmse = np.inf
rmses = []

for x in grid:
    # compute predicted win% for exponent x
    with np.errstate(over='ignore', invalid='ignore'):
        RSx = np.power(RS, x)
        RAx = np.power(RA, x)
        pred = RSx / (RSx + RAx)
    cur_rmse = rmse(WPCT, pred)
    rmses.append(cur_rmse)
    if np.isfinite(cur_rmse) and cur_rmse < best_rmse:
        best_rmse = cur_rmse
        best_x = x

print(f"Grid search best exponent: {best_x:.2f} with RMSE = {best_rmse:.5f}")

# compare to MLB baseline 1.83
mlb_x = 1.83
RSx = np.power(RS, mlb_x); RAx = np.power(RA, mlb_x)
mlb_pred = RSx / (RSx + RAx)
mlb_rmse = rmse(WPCT, mlb_pred)
print(f"Baseline exponent {mlb_x} RMSE = {mlb_rmse:.5f}")
print(f"Improvement % in RMSE: {((mlb_rmse - best_rmse) / mlb_rmse) * 100:.2f}")

# Compute per-year & per-conference best exponent
compute_per_conf = True
per_conf_results = {}

if compute_per_conf and "year" in df_clean.columns and "league" in df_clean.columns:
    for year, year_group in df_clean.groupby("year"):
        for conf, group in year_group.groupby("league"):
            RSy = group[RS_COL].to_numpy(dtype=float)
            RAy = group[RA_COL].to_numpy(dtype=float)
            WPCTy = group[WPCT_COL].to_numpy(dtype=float)
            
            # filter invalid games
            valid = (RSy + RAy) > 0
            RSy = RSy[valid]; RAy = RAy[valid]; WPCTy = WPCTy[valid]
            if len(RSy) < 3:  # skip tiny samples
                continue
            
            # Find best exponent
            best_x_c = None
            best_rmse_c = np.inf
            for x in grid:
                RSx = np.power(RSy, x)
                RAx = np.power(RAy, x)
                pred = RSx / (RSx + RAx)
                cur_rmse = rmse(WPCTy, pred)
                if np.isfinite(cur_rmse) and cur_rmse < best_rmse_c:
                    best_rmse_c = cur_rmse
                    best_x_c = x
            
            # MLB 1.83 RMSE
            RSx_mlb = np.power(RSy, 1.83)
            RAx_mlb = np.power(RAy, 1.83)
            pred_mlb = RSx_mlb / (RSx_mlb + RAx_mlb)
            mlb_rmse_c = rmse(WPCTy, pred_mlb)
            
            # improvement in RMSE
            improvement_pct = ((mlb_rmse_c - best_rmse_c) / mlb_rmse_c) * 100
            
            # results
            per_conf_results[(year, conf)] = (best_x_c, best_rmse_c, mlb_rmse_c, improvement_pct)

    print("Per-year per-conference top exponents:")
    for (year, conf), vals in sorted(per_conf_results.items()):
        best_x_c, best_rmse_c, mlb_rmse_c, improvement_pct = vals
        print(
            int(year),
            conf,
            (
                float(round(best_x_c, 3)),
                float(round(best_rmse_c, 4)),
                1.83,
                float(round(mlb_rmse_c, 4)),
                float(round(improvement_pct, 2))
            )
        )

# Find biggest difference from 1.83 across all (year, conference)
max_diff = -1
max_record = None

for (year, conf), (best_x_c, best_rmse_c, mlb_rmse_c, improvement_pct) in per_conf_results.items():
    diff = abs(best_x_c - 1.83)
    if diff > max_diff:
        max_diff = diff
        max_record = (year, conf, best_x_c, improvement_pct)

if max_record:
    year, conf, best_x_c, improvement_pct = max_record
    print(f"\nLargest deviation from 1.83: {year} in league {conf} with best_x = {best_x_c:.2f}, {improvement_pct:.2f}% improvement")

Using 1468 rows for fitting (non-missing RS/RA/WPCT).
[2021 2022 2023 2024 2025]
1468 rows after removing zero total-run rows.
Grid search best exponent: 1.79 with RMSE = 0.04586
Baseline exponent 1.83 RMSE = 0.04594
Improvement % in RMSE: 0.16
Per-year per-conference top exponents:
2021 ACC (1.78, 0.0404, 1.83, 0.0405, 0.12)
2021 ASUN (1.77, 0.0371, 1.83, 0.0373, 0.49)
2021 America East (1.31, 0.0396, 1.83, 0.0463, 14.44)
2021 Atlantic 10 (2.02, 0.0374, 1.83, 0.0389, 4.06)
2021 Big 12 (1.43, 0.0402, 1.83, 0.0513, 21.63)
2021 Big East (1.58, 0.0503, 1.83, 0.0532, 5.45)
2021 Big South (1.78, 0.0556, 1.83, 0.0557, 0.13)
2021 Big Ten (1.65, 0.0289, 1.83, 0.0313, 7.8)
2021 Big West (1.66, 0.0325, 1.83, 0.0354, 7.96)
2021 CAA (1.74, 0.026, 1.83, 0.0267, 2.55)
2021 CUSA (1.78, 0.059, 1.83, 0.0591, 0.18)
2021 Horizon (1.56, 0.0233, 1.83, 0.0329, 29.25)
2021 MAAC (1.67, 0.0718, 1.83, 0.0731, 1.77)
2021 MAC (1.68, 0.0447, 1.83, 0.0458, 2.29)
2021 MEAC (1.88, 0.0577, 1.83, 0.0578, 0.14)
2021 MVC

In [7]:
import numpy as np
import pandas as pd

RS_COL = "R (Batting)"
RA_COL = "R (Pitching)"
WPCT_COL = "WPCT"
GRID_MIN, GRID_MAX, GRID_STEP = 0.5, 3.5, 0.01

def rmse(y_true, y_pred):
    mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
    if mask.sum() == 0:
        return np.nan
    return np.sqrt(np.mean((y_true[mask] - y_pred[mask]) ** 2))

df = pd.read_csv("ncaabb_dataset.csv")

# numeric conversion + cleaning
for c in [RS_COL, RA_COL, WPCT_COL]:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df_clean = df.dropna(subset=[RS_COL, RA_COL, WPCT_COL])

# Filter so it's only 2021-2025, matching player_stat years
if "year" in df_clean.columns:
    df_clean = df_clean[(df_clean["year"] >= 2021) & (df_clean["year"] <= 2025)]

grid = np.arange(GRID_MIN, GRID_MAX + 1e-9, GRID_STEP)

# Best exponent per year
per_year_results = {}
for year, group in df_clean.groupby("year"):
    RSy, RAy, WPCTy = (
        group[RS_COL].to_numpy(float),
        group[RA_COL].to_numpy(float),
        group[WPCT_COL].to_numpy(float),
    )
    valid = (RSy + RAy) > 0
    RSy, RAy, WPCTy = RSy[valid], RAy[valid], WPCTy[valid]
    if len(RSy) < 3:
        continue
    
    best_x, best_rmse = None, np.inf
    for x in grid:
        pred = np.power(RSy, x) / (np.power(RSy, x) + np.power(RAy, x))
        cur_rmse = rmse(WPCTy, pred)
        if np.isfinite(cur_rmse) and cur_rmse < best_rmse:
            best_rmse, best_x = cur_rmse, x
    per_year_results[year] = best_x

# Best exponent per (year, conference)
per_conf_results = {}
for (year, conf), group in df_clean.groupby(["year", "league"]):
    RSy, RAy, WPCTy = (
        group[RS_COL].to_numpy(float),
        group[RA_COL].to_numpy(float),
        group[WPCT_COL].to_numpy(float),
    )
    valid = (RSy + RAy) > 0
    RSy, RAy, WPCTy = RSy[valid], RAy[valid], WPCTy[valid]
    if len(RSy) < 3:
        continue
    
    best_x, best_rmse = None, np.inf
    for x in grid:
        pred = np.power(RSy, x) / (np.power(RSy, x) + np.power(RAy, x))
        cur_rmse = rmse(WPCTy, pred)
        if np.isfinite(cur_rmse) and cur_rmse < best_rmse:
            best_rmse, best_x = cur_rmse, x
    per_conf_results[(year, conf)] = best_x

# Differences (conference – year)
diffs = {}
for (year, conf), conf_x in per_conf_results.items():
    if year in per_year_results:
        year_x = per_year_results[year]
        diff = round(conf_x - year_x, 3)
        diffs[(year, conf)] = (year_x, conf_x, diff)

# Print
print("\nConference vs Yearly Exponent Differences:")
for (year, conf), (year_x, conf_x, diff) in sorted(diffs.items()):
    print(
        f"In {year}, the best exponent was {year_x:.2f}. "
        f"For {conf}, it was {conf_x:.2f}, which is a difference of {diff:+.2f}."
    )

print("\nAverage differences by conference:")
conf_diffs_summary = {}
for (year, conf), (_, _, diff) in diffs.items():
    conf_diffs_summary.setdefault(conf, []).append(diff)

for conf, diff_list in conf_diffs_summary.items():
    avg_diff = np.mean(diff_list)
    std_diff = np.std(diff_list)
    print(f"{conf}: avg diff = {avg_diff:+.3f}, std = {std_diff:.3f}, n={len(diff_list)}")


Conference vs Yearly Exponent Differences:
In 2021, the best exponent was 1.71. For ACC, it was 1.78, which is a difference of +0.07.
In 2021, the best exponent was 1.71. For ASUN, it was 1.77, which is a difference of +0.06.
In 2021, the best exponent was 1.71. For America East, it was 1.31, which is a difference of -0.40.
In 2021, the best exponent was 1.71. For Atlantic 10, it was 2.02, which is a difference of +0.31.
In 2021, the best exponent was 1.71. For Big 12, it was 1.43, which is a difference of -0.28.
In 2021, the best exponent was 1.71. For Big East, it was 1.58, which is a difference of -0.13.
In 2021, the best exponent was 1.71. For Big South, it was 1.78, which is a difference of +0.07.
In 2021, the best exponent was 1.71. For Big Ten, it was 1.65, which is a difference of -0.06.
In 2021, the best exponent was 1.71. For Big West, it was 1.66, which is a difference of -0.05.
In 2021, the best exponent was 1.71. For CAA, it was 1.74, which is a difference of +0.03.
In 20